In [ ]:
from silab.utils import pred_accuracy
from silab.convarchs import simple_CNN, DANN
from silab.TDDAdata import TDDA_loader
import torch
import torch.nn as nn
import torch.nn.functional as F
import scipy.io as sio
import os
import numpy as np
import pandas as pd
from torch.utils.data import *
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [ ]:
if torch.cuda.is_available():
    torch.cuda.set_device(0)

In [ ]:
PATH = 'L:/vibnet-hankooktire/TDDA_dataset/npy_data/'

## Data loading

In [ ]:
tdda = TDDA_loader(PATH, split=0.85)

In [ ]:
train, test = tdda.getDataLoader()

In [ ]:
plt.plot(tdda.getData(0)[1,:][300:600])

### Simple CNN test

In [8]:
m = simple_CNN([1,3,6],1758)

In [9]:
opt = torch.optim.Adam(m.parameters(), weight_decay= 0.001, lr=0.001)
for epoch in range(2):
    run_loss = 0
    count = 0 
    for i, data in enumerate(train):
        x, y, domain_label = data
        opt.zero_grad()       
        y_hat = m(x)
        loss = F.nll_loss(y_hat,y)
        # if sample comes from target and not source ignore the loss
        if domain_label == 0:
            count += 1
        else:
            loss.backward()
            opt.step()
            run_loss += loss.item()
        if i % 100 == 0 and i != 0:
            print('Loss: {}'.format(run_loss/(i-count)))
            print('Train accuracy on source domain: {}'.format(pred_accuracy(m, train, len(train)), False,True))
            print('Train accuracy on target domains (different rpm): {}'.format(pred_accuracy(m, train, len(train),True,False)))
            print('Test accuracy on source domain: {}'.format(pred_accuracy(m, test, len(test),False,True))) 
            print('Test accuracy on target domains (different rpm): {}'.format(pred_accuracy(m, test, len(test),True,False))) 

Loss: 3.563941751311465
Train accuracy on source domain: 0.8780308596620132
Train accuracy on target domains (different rpm): 0.28037658134745513
Test accuracy on source domain: 0.8744769874476988
Test accuracy on target domains (different rpm): 0.27454242928452577
Loss: 1.8189115282357409
Train accuracy on source domain: 1.0
Train accuracy on target domains (different rpm): 0.13092085907619888
Test accuracy on source domain: 1.0
Test accuracy on target domains (different rpm): 0.129783693843594
Loss: 1.2755681370617822
Train accuracy on source domain: 1.0
Train accuracy on target domains (different rpm): 0.1412180052956752
Test accuracy on source domain: 1.0
Test accuracy on target domains (different rpm): 0.13810316139767054
Loss: 0.9640962449778967
Train accuracy on source domain: 1.0
Train accuracy on target domains (different rpm): 0.09767578699617535
Test accuracy on source domain: 1.0
Test accuracy on target domains (different rpm): 0.09650582362728785
Loss: 0.7508830082683599
T

Test accuracy on source domain: 1.0
Test accuracy on target domains (different rpm): 0.15141430948419302
Loss: 0.0016383588314056396
Train accuracy on source domain: 1.0
Train accuracy on target domains (different rpm): 0.1503383348043542
Test accuracy on source domain: 1.0
Test accuracy on target domains (different rpm): 0.15141430948419302
Loss: 0.0014642172081525935
Train accuracy on source domain: 1.0
Train accuracy on target domains (different rpm): 0.1503383348043542
Test accuracy on source domain: 1.0
Test accuracy on target domains (different rpm): 0.15141430948419302
Loss: 0.0013467372876207976
Train accuracy on source domain: 1.0
Train accuracy on target domains (different rpm): 0.1503383348043542
Test accuracy on source domain: 1.0
Test accuracy on target domains (different rpm): 0.15141430948419302
Loss: 0.0012662205041623583
Train accuracy on source domain: 1.0
Train accuracy on target domains (different rpm): 0.1503383348043542
Test accuracy on source domain: 1.0
Test acc

## DANN 

In [ ]:
m1 = simple_CNN([1,3,12,24,3,1],1955)
m2 = simple_CNN([1,3,12,24,3,1],1955)
m3 = simple_CNN([1,3,12,24,3,1],1955)

In [ ]:
m = GradientReversal(m1,m2,m2)

In [ ]:
for p in m.parameters():
    print(p)